In [1]:
import torch
import torch.tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch.utils.data as Data
torch.manual_seed(1)

In [2]:
VOCAB_SIZE = 4346
EMBEDDING_SIZE = 200
HIDDEN_SIZE = 200
TARGET_SIZE = 2
SENTENCE_LEN = 103
BATCH_SIZE = 16

In [25]:
def get_myembed():
    myembed = []
    with open("new_vector.txt","r") as f:
        for line in f:
            words = line.split()
            for word in words:
                myembed.append(float(word))
    for i in range(EMBEDDING_SIZE):myembed.append(0)
    myembed = np.array(myembed)
    myembed = myembed.reshape((VOCAB_SIZE,EMBEDDING_SIZE))
    myembed = torch.from_numpy(myembed)
    return myembed

In [26]:
class SimpleLSTM(nn.Module):
    def __init__(self,vocab_size,embedding_size,hidden_size,target_size):
        super(SimpleLSTM,self).__init__()
        self.embed = nn.Embedding(vocab_size,embedding_size)
        self.lstm = nn.LSTM(embedding_size,hidden_size)
        self.linear = nn.Linear(hidden_size,target_size)
        self.init_embed()
    def init_embed(self):
        self.embed.weight.data.copy_(get_myembed())
    def forward(self,sentence):
        myembed = self.embed(sentence)
        output,(hidden,c) = self.lstm(myembed.view(SENTENCE_LEN,-1,EMBEDDING_SIZE))
        predict = self.linear(hidden.view(-1,HIDDEN_SIZE))
        return predict        

In [5]:
mynet = SimpleLSTM(VOCAB_SIZE,EMBEDDING_SIZE,HIDDEN_SIZE,TARGET_SIZE)
mynet.cuda()
print(mynet)

SimpleLSTM(
  (embed): Embedding(4346, 200)
  (lstm): LSTM(200, 200)
  (linear): Linear(in_features=200, out_features=2, bias=True)
)


In [6]:
def load_data():
    origin_y = []
    with open("input_label.txt","r") as f:
        for line in f:
            origin_y.append(int(line))
    origin_x = []
    with open("input_pad.txt","r") as f:
        for line in f:
            words = line.split()
            assert len(words) == SENTENCE_LEN,"sentence pad"
            for word in words:
                origin_x.append(int(word))
    numpx = np.array(origin_x)
    numpx = numpx.reshape(-1,SENTENCE_LEN)
    x = torch.from_numpy(numpx)   
    numpy = np.array(origin_y)
    y = torch.from_numpy(numpy)

In [8]:

print(x.numpy().shape)

(27792, 103)


In [9]:

print(y.numpy().shape)

(27792,)


In [10]:
torch_dataset = Data.TensorDataset(x,y)
loader = Data.DataLoader(
    dataset = torch_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 2
)

In [11]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(mynet.parameters(), lr=0.1)

In [12]:
    tp = 0
    fp = 0
    fn = 0

In [13]:
def evaluate(predict,y):
    global tp 
    global fp 
    global fn
    tp += torch.masked_select(mask = y.byte(),input = predict).sum().cpu()
    fp += torch.masked_select(mask =~y.byte(),input = predict).sum().cpu()
    fn += (~y.byte()).sum().cpu() - torch.masked_select(mask =~y.byte(),input = predict).sum().cpu()
    

In [22]:
def whole_eval():
    precision = tp.float() / (tp.float() + fp.float())
    recall = tp.float() / (tp.float() + fn.float())
    print(precision)
    print(recall)
    f1 = 2 * precision * recall / (precision + recall)
    return f1

In [15]:

his_loss = []
wrongnum = 0
totalone = 0
for epoch in range(10):
    wrongnum = 0
    totalone = 0
    whole_loss = 0.0
    print(len(loader))
    for step,(batch_x,batch_y)in enumerate(loader):
        #if(step % 100) == 0:print(step)
        out = mynet(batch_x.cuda())
        pred = torch.max(F.softmax(out),1)[1]
        wrongnum += torch.sum(pred != batch_y.cuda())
        totalone += torch.sum(pred)
        loss = loss_function(out,batch_y.cuda())
        evaluate(pred.cuda(),batch_y.cuda())
        whole_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(wrongnum)
    print(totalone)
    print(whole_loss)
    his_loss.append(whole_loss)
    #print(pred)
    #print(loss)

1737


/home/xue/anaconda2/envs/my_env/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


tensor(4033, device='cuda:0')
tensor(16, device='cuda:0')
tensor(721.8217, device='cuda:0')
1737


Process Process-3:
Process Process-4:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/xue/anaconda2/envs/my_env/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/xue/anaconda2/envs/my_env/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/xue/anaconda2/envs/my_env/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/xue/anaconda2/envs/my_env/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/xue/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/home/xue/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/home/xue/anaconda2/envs/my_env/lib/python3.6/multiprocessing/queues.py", li

KeyboardInterrupt: 

In [16]:
wrongnum = 0
totalone = 0
for step,(batch_x,batch_y)in enumerate(loader):
        if(step % 100) == 0:print(step)
        out = mynet(batch_x.cuda())
        pred = torch.max(F.softmax(out),1)[1]

        wrongnum += torch.sum(pred != batch_y.cuda())
        totalone += torch.sum(pred)

0


/home/xue/anaconda2/envs/my_env/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700


In [17]:
print(his_loss)

[tensor(721.8217, device='cuda:0')]


In [23]:
print(wrongnum)
print(totalone)
print(whole_eval())

tensor(4021, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0.1250)
tensor(1.00000e-05 *
       5.3758)
tensor(1.00000e-04 *
       1.0747)
